In [1]:
import json
import statistics
import folium
import numpy as np
import pandas as pd
import os
import sys
import logging
import warnings
from pathlib import Path
from IPython.display import Image, display

from cogo import plotting, data_prep, simulation


logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
warnings.filterwarnings('ignore')

# Analysis overview

At a high level, we're looking at rider volume of COGO bikeshare bikes. The plan of analysis I'm taking is to view this almost as a queueing problem, and to simulate inbound and outbound volume at each station over some arbitrary time period, and with respect to the simultaneous activity at every other station. Ultimately, I'm interested in identifying stations that can become bottlenecks, either by virtue of having no remaining bikes to lend, or having no available docks to which to return a bike. I chose this approach for several reasons:
  - Predicting overall ridership volume, and forecasting out to some future point, is a relatively straightforward problem. I assume it has already been solved.
  - Predicting ridership volume on a per-station basis is only marginally useful: this type of analysis is likely to ignore the fact that we are dealing with a closed system in which there are a limited and pre-determined number of bikes available, of which only a subset can be rented out from a given station at any given time. Any model that is agnostic to the capacity of a given station, or to the volume of inbound traffic it receives, is not liable to provide useful or actionable predictions. (E.g., if we predict that a 15-dock station will have 30 departures over the course of an hour, how can we respond? Do we double the capacity of the station? Do we send a van to fill any empty docks? Will the inbound volume over that time period be enough to meet the demand without any intervention?)
  - Treating this as a queueing problem allows us to integrate both seasonal changes in departure volume (hourly, daily, monthly — although this implementation only looks at ridership on an hourly basis) and a transition matrix of probabilities that a prototypical rider will transit from any station `N` to any other station `M`. This specifically solves for the fact that we are operating in a closed system with a limited number of bikes, each station having access only to an even more limited subset thereof at any given point in time. It also gives us the ability to easily take action based on simulations we run: we can easily flag cases where a station is about to run out of either available bikes or free docks, allowing us to move bikes from overfull stations to underfull ones.
  
# Exploring the data

Initially I'm just going to overlay the stations with their cumulative lifetime arrival and departure counts onto a map of downtown Columbus. If we wanted to tweak this to help intuit what our later simulations might tell us, we could look at these data on the basis of average daily arrivals, departures, and net change: this might help visually identify which stations were net consumers of or producers of bikes, and would need to be manually re-balanced more frequently. But for my purposes here, just getting a sense of where everything is geographically is enough for right now.

In [2]:
APP_ROOT = Path(os.path.realpath(os.path.expanduser(os.getcwd()))).parents[0]

cogo_data, cogo_stations = data_prep.load_datasets(APP_ROOT)
hourly_trips = data_prep.prepare_hourly_trips(cogo_data)
station_crosslinks = data_prep.build_station_interlinks(cogo_data)

In [3]:
# Really, this should be normalized to control for
# number of days a station was in service, but for my
# purposes, I'm just interested in the geographic
# distribution of stations right now
df_agg = plotting.counts_by_hexagon(df=cogo_data, resolution=9)

m_hex = plotting.choropleth_map(
    df_agg=df_agg,
    name='Departure Count',
    value_col='departure_count',
    with_legend=True)
m_hex = plotting.choropleth_map(
    df_agg=df_agg,
    name='Arrival Count',
    value_col='arrival_count',
    initial_map=m_hex,
    with_legend=True,
    kind='outlier'
)
folium.map.LayerControl('bottomright', collapsed=False).add_to(m_hex)
m_hex.save(str(APP_ROOT / 'output' / 'choropleth_counts.html'))
m_hex

# Simulate ridership volumes

For the actual analysis, I'm building out a simulation that will, starting at any arbitrary hour of the day and running for any arbitrary number of minutes, use historic ridership data to simulate arrivals to and departures from each station. To achieve this, I need to first identify, for each station:
  1. How many docks are available?
  1. How many departures are there, on average, within a given hour?
  1. What is the average time between departures within a given hour?
  1. What is the probability that a rider will take a rented bike to any other individual station `M`?

Given this information, we can then
  - For each tick:
    - For each station:
      - determine if a departure should occur (sample from geometric distribution given that station's inter-departure interval for that time of day)
      - If a departure should occur:
        - If `n > 0` bikes are available:
          - determine the destination station, based on the probabilities taken from our Station `N`:Station `M` transition matrix
          - undock a bike and assign it to a to global 'in transit' list w/estimated transit time
          - reset station's time since last depart counter (This is just used for internal tracking to verify that the geometric distribution we're sampling from is behaving reasonably)
        - If no bike is available
          - The customer is filled with a deep and lingering sadness
      - Otherwise:
        - increment station's time since last departure counter

    - For each undocked bike:
      - Decrement remaining travel time by 1 tick (1 minute)
      - If remaining travel time <= 0:
        - If `n > 0` available docks at destination station:
          - dock the bike
        - Otherwise:
          - The customer is filled with a deep and lingering sadness
          - The bike is thrown into the Olentangy, never to be seen again

In [4]:
orchestrator = simulation.Orchestrator(
    cogo_stations,
    station_crosslinks,
    hourly_trips,
    bike_count=408  # max bike id in COGO data
)

In [5]:
orchestrator.run_simulation(
    start_hour=12,
    num_ticks=120)

We can then inspect all of the delays caused by over- or underfull bike stations:

In [ ]:
orchestrator.delays

# Model evaluation

To evaluate this model's performance, we can just do a straight comparison of the actual arrival and departure counts[1] predicted by our simulation on an hour-by-station basis against the average volumes of the same from our historical data.[2] I'll primarily be judging model accuracy on the basis of Median Absolute Difference (as small of a scale as we're operating on, looking at percentage differences is not super meaningful: if I simulate 5 departures for a station in an hour, but the historic average is 2, that's an 80-ish percent difference for a small delta in absolute terms).


[1] — You'll see below that I am restricting the dataset of historical observations to hours 12 and 13. This is purely a function of the constraints I specified above in terms of the starting hour of the simulation and the number of ticks to advance. You are welcome to adjust as you see fit to model a wider range of time.
[2] — Note that we could also hold an out-of-band sample to generate actual predictions against; however, given that this model is just intended to prove the concept and does not have important seasonalities integrated (day of week, month of year), that would only be of limited use and so I will decline to do so here.

In [54]:
sim_res = pd.DataFrame()
for _, group in orchestrator.simulation_statistics.groupby(['station_id']):
    group['delta'] = group['available_docks'] - group['available_docks'].shift(-1)
    sim_res = pd.concat([sim_res, group])
sim_res['departures'] = np.abs(sim_res['delta'].apply(lambda x: min(0, x)))
sim_res['arrivals'] = sim_res['delta'].apply(lambda x: max(0, x))

sim_res['hour'] = sim_res['timestamp'].apply(lambda x: x[:2])
sim_res = sim_res.groupby(['hour', 'station_id']).agg('sum').reset_index()

In [55]:
cogo_data['arrival_hour'] = cogo_data['stop_timestamp'].dt.strftime('%H')
cogo_data['arrival_date'] = cogo_data['stop_timestamp'].dt.strftime('%Y-%m-%d')

cogo_data['departure_hour'] = cogo_data['start_timestamp'].dt.strftime('%H')
cogo_data['departure_date'] = cogo_data['start_timestamp'].dt.strftime('%%Y-%m-%d')

out = (
    cogo_data
    .groupby(['arrival_hour', 'stop_station_id'])
    .agg({
        'bike_id': 'count',
        'arrival_date': pd.Series.nunique
    })
    .rename_axis(['hour', 'station_id'])
    .rename({
        'bike_id': 'arrival_count',
        'arrival_date': 'n_arrival_days'
    }, axis=1)
)
out = out.join(
    cogo_data
    .groupby(['departure_hour', 'start_station_id'])
    .agg({
        'bike_id': 'count',
        'departure_date': pd.Series.nunique
    })
    .rename_axis(['hour', 'station_id'])
    .rename({
        'bike_id': 'departure_count',
        'departure_date': 'n_departure_days'
    }, axis=1)
)
out['average_arrivals'] = round(
    out['arrival_count'] / out['n_arrival_days'], 0)
out['average_departures'] = round(
    out['departure_count'] / out['n_departure_days'], 0)
out = out.reset_index()
out = out.loc[out['hour'].isin(['12', '13'])]

In [74]:
final = sim_res.merge(out, on=['hour', 'station_id'], how='outer')
# final['arrival_deviance'] = (
#     np.abs(final['arrivals'] - final['average_arrivals']) / (
#         (final['arrivals'] + final['average_arrivals']) / 2
#     ) * 100
# )
# final['departure_deviance'] = (
#     np.abs(final['departures'] - final['average_departures']) / (
#         (final['departures'] + final['average_departures']) / 2
#     ) * 100
# )

final['arrival_delta'] = np.abs(final['arrivals'] - final['average_arrivals'])
final['departure_delta'] = np.abs(final['departures'] - final['average_departures'])

final.agg({
    'departure_delta': ['mean', 'median'],
    'arrival_delta': ['mean', 'median']
})

,departure_delta,arrival_delta
mean,1.314286,0.871429
median,1.000000,1.000000


For both arrivals and departures, by both mean and median absolute difference, my simulation is on average within about a single departure or arrival of actuals.